In [1]:
import sys
sys.executable

'/Users/eranagmon/code/process-bigraph/venv/bin/python3'

In [2]:
import numpy as np
from process_bigraph import Process, ProcessTypes, Composite
import matplotlib.pyplot as plt
import cobra
from cobra.io import load_model
from scipy.ndimage import convolve

core = ProcessTypes()


In [3]:
# create new types
def apply_non_negative(schema, current, update, core):
    new_value = current + update
    return max(0, new_value)

positive_float = {
    '_type': 'positive_float',
    '_inherit': 'float',
    '_apply': apply_non_negative
}
core.register('positive_float', positive_float)

bounds_type = {
    'lower': 'maybe[float]',
    'upper': 'maybe[float]'
}
core.register_process('bounds', bounds_type)

In [4]:
core.access('positive_float')

{'_type': 'positive_float',
 '_check': 'check_float',
 '_apply': 'apply_non_negative',
 '_serialize': 'to_string',
 '_description': '64-bit floating point precision number',
 '_default': '0.0',
 '_deserialize': 'deserialize_float',
 '_divide': 'divide_float',
 '_dataclass': 'dataclass_float',
 '_inherit': ['float']}

## Dynamic FBA Process

In [5]:
class DynamicFBA(Process):
    """
    Performs dynamic FBA.

    Parameters:
    - model: The metabolic model for the simulation.
    - kinetic_params: Kinetic parameters (Km and Vmax) for each substrate.
    - biomass_reaction: The identifier for the biomass reaction in the model.
    - substrate_update_reactions: A dictionary mapping substrates to their update reactions.
    - biomass_identifier: The identifier for biomass in the current state.

    TODO -- check units
    """

    config_schema = {
        'model_file': 'string',
        'kinetic_params': 'map[tuple[float,float]]',
        'biomass_reaction': {
            '_type': 'string',
            '_default': 'Biomass_Ecoli_core'
        },
        'substrate_update_reactions': 'map[string]',
        'biomass_identifier': 'string',
        'bounds': 'map[bounds]',
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        if not 'xml' in self.config['model_file']:
            # use the textbook model if no model file is provided
            self.model = load_model(self.config['model_file'])
        else:
            self.model = cobra.io.read_sbml_model(self.config['model_file'])

        for reaction_id, bounds in self.config['bounds'].items():
            if bounds['lower'] is not None:
                self.model.reactions.get_by_id(reaction_id).lower_bound = bounds['lower']
            if bounds['upper'] is not None:
                self.model.reactions.get_by_id(reaction_id).upper_bound = bounds['upper']

    def inputs(self):
        return {
            'substrates': 'map[positive_float]'
        }

    def outputs(self):
        return {
            'substrates': 'map[positive_float]'
        }

    # TODO -- can we just put the inputs/outputs directly in the function?
    def update(self, state, interval):
        substrates_input = state['substrates']

        for substrate, reaction_id in self.config['substrate_update_reactions'].items():
            Km, Vmax = self.config['kinetic_params'][substrate]
            substrate_concentration = substrates_input[substrate]
            uptake_rate = Vmax * substrate_concentration / (Km + substrate_concentration)
            self.model.reactions.get_by_id(reaction_id).lower_bound = -uptake_rate

        substrate_update = {}

        solution = self.model.optimize()
        if solution.status == 'optimal':
            current_biomass = substrates_input[self.config['biomass_identifier']]
            biomass_growth_rate = solution.fluxes[self.config['biomass_reaction']]
            substrate_update[self.config['biomass_identifier']] = biomass_growth_rate * current_biomass * interval

            for substrate, reaction_id in self.config['substrate_update_reactions'].items():
                flux = solution.fluxes[reaction_id]
                substrate_update[substrate] = flux * current_biomass * interval
                # TODO -- assert not negative?
        else:
            # Handle non-optimal solutions if necessary
            # print('Non-optimal solution, skipping update')
            for substrate, reaction_id in self.config['substrate_update_reactions'].items():
                substrate_update[substrate] = 0

        return {
            'substrates': substrate_update,
        }

core.register_process('DynamicFBA', DynamicFBA)

In [6]:
from process_bigraph.experiments.parameter_scan import RunProcess

def dfba_config(
        model_file='textbook',
        kinetic_params={
            'glucose': (0.5, 1),
            'acetate': (0.5, 2)},
        biomass_reaction='Biomass_Ecoli_core',
        substrate_update_reactions={
            'glucose': 'EX_glc__D_e',
            'acetate': 'EX_ac_e'},
        biomass_identifier='biomass',
        bounds={
            'EX_o2_e': {'lower': -2, 'upper': None},
            'ATPM': {'lower': 1, 'upper': 1}}
):
    return {
        'model_file': model_file,
        'kinetic_params': kinetic_params,
        'biomass_reaction': biomass_reaction,
        'substrate_update_reactions': substrate_update_reactions,
        'biomass_identifier': biomass_identifier,
        'bounds': bounds
    }


# TODO -- this should be imported, or just part of Process?
def run_process(
        address,
        config,
        core_type,
        initial_state,
        observables,
        timestep=1,
        runtime=10
):
    config = {
        'process_address': address,
        'process_config': config,
        'observables': observables,
        'timestep': timestep,
        'runtime': runtime}

    run = RunProcess(config, core_type)
    return run.update(initial_state)

In [7]:
n_bins = (5, 5)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

dfba_processes_dict = {}
for i in range(n_bins[0]):
    for j in range(n_bins[1]):
        dfba_processes_dict[f'[{i},{j}]'] = {
            '_type': 'process',
            'address': 'local:DynamicFBA',
            'config': dfba_config(),
            'inputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j],
                }
            },
            'outputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j]
                }
            }
        }

composite_state = {
    'fields': {
        '_type': 'map',
        '_value': {
            '_type': 'array',
            '_shape': n_bins,
            '_data': 'positive_float'
        },
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'spatial_dfba': dfba_processes_dict,
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'fields': 'map',
                'time': 'float',
            }
        },
        'inputs': {
            'fields': ['fields'],
            'time': ['global_time']
        }
    }
}

sim = Composite({'state': composite_state}, core=core)

sim.update({}, 10.0)



[]

In [8]:
results = sim.gather_results()[('emitter',)]
results

[{'fields': {'_type': 'map',
   '_value': {'_type': 'array', '_shape': (5, 5), '_data': 'positive_float'},
   'glucose': array([[ 1.05494692,  3.21135777,  7.62934694, 18.13914579, 19.5392402 ],
          [14.3321519 , 19.17272223, 13.75329196, 19.95907999, 15.48155216],
          [17.34048488,  6.20481708,  8.71169331, 10.15838692,  5.57509712],
          [10.76296875, 13.49779122,  8.41667   , 13.88647257, 17.98651318],
          [15.87549511,  4.4793677 , 16.28180449, 15.1458885 ,  6.9025743 ]]),
   'acetate': array([[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]),
   'biomass': array([[0.00403388, 0.07873172, 0.09651373, 0.0857904 , 0.09322035],
          [0.09099417, 0.05576586, 0.06044433, 0.09543499, 0.08556801],
          [0.00485139, 0.05965545, 0.07130176, 0.03765414, 0.0931261 ],
          [0.0540483 , 0.03666924, 0.01203241, 0.01498518, 0.07911831],
          [0.08204151, 

In [9]:
# plot results
def plot_results(results):
    fig, ax = plt.subplots()
    for key, value in results['substrates'].items():
        ax.plot(results['time'], value, label=key)
    ax.legend()
    plt.show()

In [10]:
# plot_results(results['results'])

TypeError: list indices must be integers or slices, not str

## Diffusion Advection Process

In [11]:

# Laplacian for 2D diffusion
LAPLACIAN_2D = np.array([[0, 1, 0],
                         [1, -4, 1],
                         [0, 1, 0]])


class DiffusionAdvection(Process):
    config_schema = {
        'n_bins': 'tuple[integer,integer]',
        'bounds': 'tuple[float,float]',
        'default_diffusion_rate': {'_type': 'float', '_default': 1e-1},
        'default_diffusion_dt': {'_type': 'float', '_default': 1e-1},
        'diffusion_coeffs': 'map[float]',
        'advection_coeffs': 'map[tuple[float,float]]',
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        # get diffusion rates
        bins_x = self.config['n_bins'][0]
        bins_y = self.config['n_bins'][1]
        length_x = self.config['bounds'][0]
        length_y = self.config['bounds'][1]
        dx = length_x / bins_x
        dy = length_y / bins_y
        dx2 = dx * dy

        # general diffusion rate
        diffusion_rate = self.config['default_diffusion_rate']
        self.diffusion_rate = diffusion_rate / dx2

        # diffusion rates for each individual molecules
        self.molecule_specific_diffusion = {
            mol_id: diff_rate / dx2
            for mol_id, diff_rate in self.config['diffusion_coeffs'].items()}

        # get diffusion timestep
        diffusion_dt = 0.5 * dx ** 2 * dy ** 2 / (2 * diffusion_rate * (dx ** 2 + dy ** 2))
        self.diffusion_dt = min(diffusion_dt, self.config['default_diffusion_dt'])

    def inputs(self):
        return {
            'fields': {
                '_type': 'map',
                '_value': {
                    '_type': 'array',
                    '_shape': self.config['n_bins'],
                    '_data': 'positive_float'
                },
            }
        }

    def outputs(self):
        return {
            'fields': {
                '_type': 'map',
                '_value': {
                    '_type': 'array',
                    '_shape': self.config['n_bins'],
                    '_data': 'positive_float'
                },
            }
        }

    def update(self, state, interval):
        fields = state['fields']

        fields_update = {}
        for species, field in fields.items():
            fields_update[species] = self.diffusion_delta(
                field,
                interval,
                diffusion_coeff=self.config['diffusion_coeffs'][species],
                advection_coeff=self.config['advection_coeffs'][species]
            )

        return {
            'fields': fields_update
        }

    def diffusion_delta(self, state, interval, diffusion_coeff, advection_coeff):
        t = 0.0
        dt = min(interval, self.diffusion_dt)
        updated_state = state.copy()

        while t < interval:

            # Diffusion
            laplacian = convolve(
                updated_state,
                LAPLACIAN_2D,
                mode='reflect',
            ) * diffusion_coeff

            # Advection
            advective_flux_x = convolve(
                updated_state,
                np.array([[-1, 0, 1]]),
                mode='reflect',
            ) * advection_coeff[0]
            advective_flux_y = convolve(
                updated_state,
                np.array([[-1], [0], [1]]),
                mode='reflect',
            ) * advection_coeff[1]

            # Update the current state
            updated_state += (laplacian + advective_flux_x + advective_flux_y) * dt

            # # Ensure non-negativity
            # current_states[species] = np.maximum(updated_state, 0)

            # Update time
            t += dt

        return updated_state - state

core.register_process('DiffusionAdvection', DiffusionAdvection)

In [12]:
 n_bins = (4, 4)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

composite_state = {
    'fields': {
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'diffusion': {
        '_type': 'process',
        'address': 'local:DiffusionAdvection',
        'config': {
            'n_bins': n_bins,
            'bounds': (10, 10),
            'default_diffusion_rate': 1e-1,
            'default_diffusion_dt': 1e-1,
            'diffusion_coeffs': {
                'glucose': 1e-1,
                'acetate': 1e-1,
                'biomass': 1e-1,
            },
            'advection_coeffs': {
                'glucose': (0, 0),
                'acetate': (0, 0),
                'biomass': (0, 0),
            },
        },
        'inputs': {
            'fields': ['fields']
        },
        'outputs': {
            'fields': ['fields']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'fields': 'map',
                'time': 'float',
            }
        },
        'inputs': {
            'fields': ['fields'],
            'time': ['global_time'],
        }
    }
}

sim = Composite({'state': composite_state}, core=core)
# sim.add_emitter()

sim.update({}, 10.0)

data = sim.gather_results()[('emitter',)]

print(data)

[{'fields': {'glucose': array([[11.49673574,  0.68256312, 18.54146763,  0.95746649],
       [ 3.19256133,  7.66018853,  5.83104959, 15.26516279],
       [11.02857073,  8.32815473, 13.70634628,  3.93032842],
       [17.69033152, 17.09328189,  9.9747383 ,  3.19076489]]), 'acetate': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), 'biomass': array([[0.05699761, 0.09481982, 0.02570525, 0.02258643],
       [0.01204724, 0.01354382, 0.08648323, 0.00512884],
       [0.04747683, 0.04636961, 0.06420193, 0.01107399],
       [0.02910536, 0.04286344, 0.03248139, 0.09196548]])}, 'time': 0.0}, {'fields': {'glucose': array([[ 9.84321673,  3.73152289, 14.39153868,  3.76520899],
       [ 5.00816707,  6.9420476 ,  8.21497622, 12.28942337],
       [10.77908124,  9.56199548, 11.56953882,  5.64172408],
       [16.88162215, 15.68484685, 10.22024593,  4.04455588]]), 'acetate': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
      

## COMETS

In [13]:
n_bins = (5, 5)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

dfba_processes_dict = {}
for i in range(n_bins[0]):
    for j in range(n_bins[1]):
        dfba_processes_dict[f'[{i},{j}]'] = {
            '_type': 'process',
            'address': 'local:DynamicFBA',
            'config': dfba_config(),
            'inputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j],
                }
            },
            'outputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j]
                }
            }
        }

composite_state = {
    'fields': {
        '_type': 'map',
        '_value': {
            '_type': 'array',
            '_shape': n_bins,
            '_data': 'positive_float'
        },
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'spatial_dfba': dfba_processes_dict,
    'diffusion': {
        '_type': 'process',
        'address': 'local:DiffusionAdvection',
        'config': {
            'n_bins': n_bins,
            'bounds': (10, 10),
            'default_diffusion_rate': 1e-1,
            'default_diffusion_dt': 1e-1,
            'diffusion_coeffs': {
                'glucose': 1e-1,
                'acetate': 1e-1,
                'biomass': 1e-1,
            },
            'advection_coeffs': {
                'glucose': (0, 0),
                'acetate': (0, 0),
                'biomass': (0, 0),
            },
        },
        'inputs': {
            'fields': ['fields']
        },
        'outputs': {
            'fields': ['fields']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'fields': 'map',
                'time': 'float',
            }
        },
        'inputs': {
            'fields': ['fields'],
            'time': ['global_time']
        }
    }
}

sim = Composite({'state': composite_state}, core=core)

sim.update({}, 10.0)


KeyError: '_type'

In [ ]:
result

## Spatial dFBA composite simulation

In [ ]:
n_bins = (2, 2)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

dfba_processes_dict = {}
for i in range(n_bins[0]):
    for j in range(n_bins[1]):
        dfba_processes_dict[f'[{i},{j}]'] = {
            '_type': 'process',
            'address': 'local:DynamicFBA',
            'config': dfba_config(),
            'inputs': {
                'substrates': {
                    'glucose': ['..', '..', 'fields', 'glucose', i, j],
                    'acetate': ['..', '..', 'fields', 'acetate', i, j],
                    'biomass': ['..', '..', 'fields', 'biomass', i, j],
                }
            },
            'outputs': {
                'substrates': {
                    'glucose': ['..', '..', 'fields', 'glucose', i, j],
                    'acetate': ['..', '..', 'fields', 'acetate', i, j],
                    'biomass': ['..', '..', 'fields', 'biomass', i, j]
                }
            }
        }

composite_state = {
    'fields': {
        '_type': 'map',
        '_value': {
            '_type': 'array',
            '_size': n_bins,
            '_value': 'positive_float'
        },
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'spatial_dfba': dfba_processes_dict
}

sim = Composite({'state': composite_state}, core=core)


In [ ]:
sim.state['spatial_dfba']

In [ ]:
composite_state